# Installation

In [1]:
# !pip install boto3

# Create SQS Standard Resource

In [2]:
# initialize sqs client
import boto3
sqs_client = boto3.client('sqs', region_name='us-east-1')

In [3]:
# Define Standard Queue Attributes
queue_name = 'dsslee-std-queue'
print(f"Queue will be made with the name: {queue_name}")

attributes = {
    'DelaySeconds': '0',                # No delay for message processing (default: 0)
    'MaximumMessageSize': '262144',     # Maximum message size (default: 262144 -> 256 KB)
    'MessageRetentionPeriod': '345600', # Message retention period (default: 345600s -> 4days)
    'ReceiveMessageWaitTimeSeconds': '0', # Wait up to n seconds for messages to be available (default: 0)
}

Queue will be made with the name: dsslee-std-queue


In [4]:
# create Standard queue
response = sqs_client.create_queue(
    QueueName = queue_name,
    Attributes = attributes
)

# get url of newly created Standard queue
queue_url = response['QueueUrl']
print("Standard queue created successfully with URL:", queue_url)

Standard queue created successfully with URL: https://sqs.us-east-1.amazonaws.com/160243126421/dsslee-std-queue


# Making function to include all standard attributes

In [1]:
def create_standard_queue(queue_name, message_retention_period=None, visibility_timeout=None,
                          maximum_message_size=None, receive_message_wait_time=None,
                          redrive_policy=None, **kwargs):
    """
    Create a standard queue in Amazon SQS with customizable attributes.

    Args:
    - queue_name: Name of the standard queue to be created.
    - message_retention_period: Message retention period in seconds (optional).
    - visibility_timeout: Visibility timeout in seconds (optional).
    - maximum_message_size: Maximum message size in bytes (optional).
    - receive_message_wait_time: Receive message wait time in seconds (optional).
    - redrive_policy: Redrive policy for handling failed messages (optional).
    - **kwargs: Additional custom attributes for the standard queue.

    Returns:
    - queue_url: URL of the newly created standard queue, or None if queue already exists.
    """

    # Initialize SQS client
    sqs_client = boto3.client('sqs')

    try:
        # Check if the queue already exists
        response = sqs_client.get_queue_url(QueueName=queue_name)
        queue_url = response['QueueUrl']
        print(f"Queue '{queue_name}' already exists.")
        return None

    except ClientError as e:
        if e.response['Error']['Code'] != 'AWS.SimpleQueueService.NonExistentQueue':
            raise

    # Define default attributes for standard queue
    attributes = {}

    # Add optional attributes if provided
    optional_attributes = {
        'MessageRetentionPeriod': message_retention_period,
        'VisibilityTimeout': visibility_timeout,
        'MaximumMessageSize': maximum_message_size,
        'ReceiveMessageWaitTimeSeconds': receive_message_wait_time,
        'RedrivePolicy': redrive_policy
    }

    for attr_name, attr_value in optional_attributes.items():
        if attr_value is not None:
            attributes[attr_name] = str(attr_value)

    # Add any additional custom attributes
    attributes.update(kwargs)

    # Create the standard queue
    response = sqs_client.create_queue(
        QueueName=queue_name,
        Attributes=attributes
    )

    # Get the URL of the newly created standard queue
    queue_url = response['QueueUrl']

    return queue_url

In [2]:
# EXAMPLE USAGE

# initialize SQS client
import boto3
from botocore.exceptions import ClientError

region_name = 'us-east-1' # Replace with your AWS region
sqs_client = boto3.client('sqs', region_name=region_name)

# Example usage:
# Example usage:
queue_name = 'dsslee-standard-queue'
message_retention_period = 86400  # 1 day message retention period
visibility_timeout = 30  # 30 seconds visibility timeout
maximum_message_size = 262144  # Maximum message size of 256 KB
receive_message_wait_time = 20  # 20 seconds receive message wait time

# Create standard queue with custom attributes
queue_url = create_standard_queue(queue_name,
                                  message_retention_period=message_retention_period,
                                  visibility_timeout=visibility_timeout,
                                  maximum_message_size=maximum_message_size,
                                  receive_message_wait_time=receive_message_wait_time,
                                  )

if queue_url:
    print("Standard queue created successfully with URL:", queue_url)
else:
    print("Failed to create standard queue.")



Standard queue created successfully with URL: https://sqs.us-east-1.amazonaws.com/160243126421/dsslee-standard-queue


In [3]:
# Trying to create standard queue with same name:
queue_url = create_standard_queue(queue_name,
                                  message_retention_period=message_retention_period,
                                  visibility_timeout=visibility_timeout,
                                  maximum_message_size=maximum_message_size,
                                  receive_message_wait_time=receive_message_wait_time,
                                  )

if queue_url:
    print("Standard queue created successfully with URL:", queue_url)
else:
    print("Failed to create standard queue.")

Queue 'dsslee-standard-queue' already exists.
Failed to create standard queue.
